In [1]:
import numpy as np 
import pandas as pd 
import warnings
warnings.filterwarnings('ignore')
import os

from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import psycopg2
import sqlalchemy
from sqlalchemy import create_engine

from psycopg2.extensions import register_adapter, AsIs

def addapt_numpy_float64(numpy_float64):
    return AsIs(numpy_float64)

def addapt_numpy_int64(numpy_int64):
    return AsIs(numpy_int64)

register_adapter(np.float64, addapt_numpy_float64)
register_adapter(np.int64, addapt_numpy_int64)

In [2]:
from config import yelp_api_key, darksky_api_key, PGHOST, PGDATABASE, PGUSER, PGPASSWORD
from restaurant_info import Location
from weather import Weather

### Get Latitude & Longitude from Yelp API

In [3]:
search_business = 'The Counting Room' # Not the actual Restaurant 
location = 'Brooklyn, NY'

In [4]:
# Make Yelp API Call to get Latitude & Longitude for Business
loc = Location(search_business, location)
lat, long = loc.lat_long()

### Import / Clean / Prep File

In [3]:
w_start = '2017-01-01'
w_end = '2019-06-30'

# Restaurant File
sales_file = 'csv/rest_1_both_pos_by_check.csv'

# Complete Daily Reservations File
resy_file = 'csv/rest_1_covers_new.csv'

# Weather File
weather_csv_file = f'csv/weather_{w_start}_to_{w_end}.csv'

In [6]:
 # Read in Sales File
data = pd.read_csv(sales_file, index_col = 'date', parse_dates=True)
sales_df = pd.DataFrame(data)

In [7]:
def prep_sales_df(df):
    
        # Dinner Only
        df = df[df.day_part == 'Dinner']
        
        # Fill NaN
        df.fillna(0, inplace=True)
        
        # Filter Out $0 Sales
        df = df[df.net_sales > 0]
        
        # Create Unique Check ID
        df['check_id'] = df.index.strftime('%Y%m%d') + '-' + df.index.strftime('%H%M') + '-' + \
                    df['check_no'].astype(int).astype(str) + '-' + df['revenue_center'].str[0] + '-' + pd.factorize(df['net_sales'])[0].astype(str)
         
            
        df.check_no = df.check_no.astype(int)
        
        # Drop Covers, Day Part, & Check_No
        df = df.drop(['covers', 'day_part'], axis=1)
    
        df.index = df.index.normalize()
        
        return df
    
sales_df = prep_sales_df(sales_df)

In [8]:
sales_df.head()

,check_no,net_sales,revenue_center,check_id
date,,,,
2017-01-02,3364,7.34,Window,20170102-0511-3364-W-0
2017-01-02,3367,30.00,Window,20170102-0525-3367-W-1
2017-01-02,3369,111.00,PDR,20170102-0528-3369-P-2
2017-01-02,3370,112.00,Dining Room,20170102-0532-3370-D-3
2017-01-02,3371,148.00,Dining Room,20170102-0533-3371-D-4


In [9]:
# Send this Long Format File to CSV for Dashboards
sales_df.to_csv('csv/rest_1_sales_by_check_long_0117_0619.csv')

### High Level View of Annual Revenue

In [14]:
sales_df.groupby([sales_df.index.year, 'revenue_center'])['net_sales'].agg({'sum': 'sum', 'check_avg': 'mean'})

sum   check_avg
date revenue_center                        
2017 Bar             1080561.72  130.047144
     Dining Room     2800349.31  228.655941
     Outside          866862.75  154.962951
     PDR             1210661.82  219.521636
     Window           151524.71   34.865327
2018 Bar             1074150.34  134.470498
     Dining Room     2791638.47  216.389309
     Outside          759544.82  165.658630
     PDR             1225295.67  237.967697
     Window            98340.87   31.651390
2019 Bar              551353.66  136.676663
     Dining Room     1562307.41  199.860229
     Outside          240349.88  169.379760
     PDR              616058.02  227.243829
     Window             9725.54   14.302265

In [15]:
dfw = pd.read_csv(weather_csv_file, index_col='date', parse_dates=True)

In [16]:
dfw.head()

,apparent_temperature,humidity,precip_intensity_max,precip_max_time,precip_prob,precip_type,pressure,summary,temperature
date,,,,,,,,,
2017-01-01,37.83,0.54,0.0000,05:01AM,0.00,none,1028.26,clear-night,37.83
2017-01-02,35.58,0.92,0.0242,03:00PM,0.79,rain,1027.98,rain,39.06
2017-01-03,41.50,0.94,0.0913,05:00PM,0.77,rain,1000.08,rain,43.19
2017-01-04,36.81,0.40,0.0176,12:00AM,0.00,rain,1002.55,clear-night,42.29
2017-01-05,27.11,0.48,0.0000,05:01AM,0.00,none,1014.61,cloudy,31.36


In [17]:
dfr = pd.read_csv(resy_file, index_col='date', parse_dates=True)

def prep_resy_df(df):
                  
        # Fill NaN
        df.fillna(0, inplace=True)
        
        return df
    
dfr = prep_resy_df(dfr)

In [18]:
rental_fees = pd.read_csv('csv/pos_1_rental_fees.csv')

In [19]:
rental_fees.head()

,Ordered at,Revenue Center,Check #,Net Sales
0,10/1/2018,Dining Room,1256,"$15,000.00"
1,10/1/2018,PDR,5653,"$15,000.00"
2,1/26/2019,Dining Room,9117,"$15,000.00"
3,5/20/2019,PDR,7729,"$15,000.00"
4,9/29/2018,PDR,6436,"$2,000.00"


In [20]:
def clean_rental_fees(df):
    
    df.columns = ['date', 'rev_center', 'check_no', 'rental_fees']
    df.date = pd.to_datetime(df.date)
    
    df.rental_fees = [float(x.strip('$').replace(',', '')) for x in df.rental_fees]
    
    df.rev_center = df.rev_center.replace('Patio', 'Outside')
    
    df = df.set_index('date')
    
    return df

rental_fees = clean_rental_fees(rental_fees)

In [21]:
rental_fees.head()

,rev_center,check_no,rental_fees
date,,,
2018-10-01,Dining Room,1256,15000.0
2018-10-01,PDR,5653,15000.0
2019-01-26,Dining Room,9117,15000.0
2019-05-20,PDR,7729,15000.0
2018-09-29,PDR,6436,2000.0


## Load Data to AWS RDS

### Connect Helper Function

In [22]:
def connect():
    
    # Set up a connection to the postgres server.
    conn_string = "host="+ PGHOST +" port="+ "5432" +" dbname="+ PGDATABASE +" user=" + PGUSER \
                  +" password="+ PGPASSWORD
    
    conn = psycopg2.connect(conn_string)
    print(":) :) :)")

    # Create a cursor object
    cursor = conn.cursor()
    
    return conn, cursor

In [21]:
conn, cursor = connect()

:) :) :)


In [10]:
conn.close()

### Drop Table Helper Function

In [22]:
def drop_table(table_name):
    
    conn, cursor = connect()
    
    cursor.execute(f'DROP TABLE {table_name}')
    conn.commit()
    conn.close()

In [23]:
drop_table('weather')

:) :) :)


### Create Sales & Reservations Tables

In [5]:
create_rev_center_table = """
    CREATE TABLE IF NOT EXISTS rev_center(
        id INTEGER PRIMARY KEY NOT NULL,
        name VARCHAR NOT NULL
    )"""

create_check_table = """
    CREATE TABLE IF NOT EXISTS checks(
        check_id VARCHAR(36) PRIMARY KEY NOT NULL,
        date DATE,
        check_no INTEGER,
        rev_center_id INTEGER REFERENCES rev_center(id),
        net_sales NUMERIC (7, 2)
    )"""

create_resy_table = """
    CREATE TABLE IF NOT EXISTS reservations (
        id INTEGER PRIMARY KEY NOT NULL,
        date DATE,
        inside_covers INTEGER,
        outside_covers INTEGER,
        reserved_covers INTEGER,
        walkin_covers INTEGER,
        waitlist_covers INTEGER,
        no_show_covers INTEGER,
        no_show_parties INTEGER
    )
    """

create_rental_fees_table = """
    CREATE TABLE IF NOT EXISTS rental_fees (
        check_no INTEGER PRIMARY KEY,
        date DATE,
        rev_center_id INTEGER REFERENCES rev_center(id),
        rental_fees NUMERIC(7, 2)
    )"""

create_weather_table = """
        CREATE TABLE IF NOT EXISTS weather ( 
            DATE DATE, 
            date_id INT PRIMARY KEY, 
            apparent_temperature NUMERIC (4, 2), 
            humidity NUMERIC (3, 2), 
            precip_intensity_max NUMERIC (5, 4), 
            precip_max_time TIME, 
            precip_prob NUMERIC (3, 2), 
            precip_type TEXT, 
            pressure NUMERIC (6,2), 
            summary TEXT, 
            temperature NUMERIC (4, 2)
            )
            """

### Create Table Helper Function

In [6]:
def create_table(create_query):
    
    conn, cursor = connect()
    
    cursor.execute(f'{create_query}')
    conn.commit()
    conn.close()

In [26]:
create_table(create_weather_table)

:) :) :)


### Populate Databases

In [640]:
rev_centers = {'Dining Room': 1, 'Bar': 2, 'PDR': 3, 'Window': 4, 'Outside': 5}

In [642]:
def populate_rev_center(df):
    
    conn, cursor = connect()
    
    for r, i in rev_centers.items():
        cursor.execute("INSERT INTO rev_center (id, name) VALUES (%s, %s)", (i, r) )
        conn.commit()
        conn.close()
    
populate_rev_center(sales_df)
                

In [551]:
def populate_checks(df):
    
    conn, cursor = connect()
    
    for row in range(len(df)):
        
        cursor.execute("INSERT INTO checks (check_id, date, check_no, rev_center_id, net_sales) VALUES (%s, %s, %s, %s, %s)",
                      (df.iloc[row]['check_id'],
                       pd.to_datetime(df.index[row]),
                       df.iloc[row]['check_no'],
                       rev_centers[df.iloc[row]['revenue_center']],
                       df.iloc[row]['net_sales']) )
        conn.commit()
        conn.close()
        
populate_checks(sales_df)

In [661]:
def populate_reservations(df):
    
    conn, cursor = connect()
    
    for row in range(len(df)):
        
        cursor.execute("""
            INSERT INTO reservations (id, date, inside_covers, outside_covers, reserved_covers,
            walkin_covers, waitlist_covers, no_show_covers, no_show_parties) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)""",
                (df.index[row].strftime('%Y%m%d'),
                pd.to_datetime(df.index[row]),
                df.iloc[row]['inside_covers'],
                df.iloc[row]['outside_covers'],
                df.iloc[row]['reserved_covers'],
                df.iloc[row]['walkin_covers'],
                df.iloc[row]['waitlist_covers'],
                df.iloc[row]['no_show_covers'],
                df.iloc[row]['no_show_parties']) )
        conn.commit()
    conn.close()
        
populate_reservations(dfr)

In [680]:
def populate_rental_fees(df):
    
    conn, cursor = connect()
    
    for row in range(len(df)):
        
        cursor.execute("INSERT INTO rental_fees (check_no, date, rev_center_id, rental_fees) VALUES (%s, %s, %s, %s)",
                      (df.iloc[row]['check_no'],
                       pd.to_datetime(df.index[row]),
                       rev_centers[df.iloc[row]['rev_center']],
                       df.iloc[row]['rental_fees']) )
        conn.commit()
    conn.close()
        
populate_rental_fees(rental_fees)

:) :) :)


In [28]:
def populate_weather(df):
    
    conn, cursor = connect()
    
    for row in range(len(df)):
        
        cursor.execute("""
            INSERT INTO weather (date, date_id, apparent_temperature, humidity, precip_intensity_max, precip_max_time,
            precip_prob, precip_type, pressure, summary, temperature) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)""",
                (pd.to_datetime(df.index[row]),
                 df.index[row].strftime('%Y%m%d'),
                 df.iloc[row]['apparent_temperature'],
                 df.iloc[row]['humidity'],
                 df.iloc[row]['precip_intensity_max'],
                 df.iloc[row]['precip_max_time'],
                 df.iloc[row]['precip_prob'],
                 df.iloc[row]['precip_type'],
                 df.iloc[row]['pressure'],
                 df.iloc[row]['summary'],
                 df.iloc[row]['temperature']) )
        conn.commit()
    conn.close()

populate_weather(dfw)
        

:) :) :)
